In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pafy
url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4") 
from google.colab.patches import cv2_imshow

In [0]:
!pip install pafy
!pip install youtube_dl

     |████████████████████████████████| 1.8MB 8.2MB/s 


In [0]:
capture = cv2.VideoCapture(play.url)
print(capture)

<VideoCapture 0x7f429b46a330>


In [0]:
capture.set(3, 480) 
capture.set(4, 640)

False

In [0]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Female', 'Male']
font = cv2.FONT_HERSHEY_SIMPLEX

In [0]:
def load_models():
    age_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/OpenCV/deploy_age.prototxt', '/content/drive/My Drive/OpenCV/age_net.caffemodel')
    gender_net = cv2.dnn.readNetFromCaffe('/content/drive/My Drive/OpenCV/deploy_gender.prototxt', '/content/drive/My Drive/OpenCV/gender_net.caffemodel')
    return(age_net,gender_net)

In [0]:
def video_detector(age_net, gender_net):
    font = cv2.FONT_HERSHEY_SIMPLEX

In [0]:
age_net,gender_net = load_models()
video_detector(age_net,gender_net)

In [0]:
while True:
    ret, image = capture.read()
    face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/OpenCV/haarcascade_frontalface_alt.xml')
 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if(len(faces)>0):
        print("Found {} faces".format(str(len(faces))))
    for (x, y, w, h )in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2) 
        face_img = image[y:y+h, x:x+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        print("Gender : " + gender)
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]
        print("Age Range: " + age)
    overlay_text = "%s %s" % (gender, age)
    cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA) 
    cv2_imshow(image)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break